In [1]:
import numpy as np
import pandas as pd


#### movies---> genre
movie_dict = {}
movie_map = {}

with open("./content/movies.dat", "r", encoding="latin1") as file:
    for line in file:
        # Split the line into MovieID, Title, and Genres
        parts = line.strip().split("::")
        movie_id = parts[0]
        title = parts[1]
        genres = parts[2].split("|")  # Split genres by "|"

        # Create a dictionary entry for the movie
        movie_dict[movie_id] = {
            'title': title,
            'genres': genres
        }
        movie_map[title] = {
            'id': movie_id
        }


In [2]:
rm = np.full((6042, 3953), np.nan)

In [3]:
with open("./content/ratings.dat", "r") as file:
    for line in file:
        # Split the line into UserID, MovieID, Rating, and Timestamp
        parts = line.strip().split("::")
        UserID = int(parts[0])
        MovieID = int(parts[1])
        Rating = float(parts[2])
        rm[UserID, MovieID] = Rating

In [4]:
def map_age(age):
    if age < 18:
        return 1
    elif age >= 18 and age < 25:
        return 18
    elif age >= 25 and age < 35:
        return 25
    elif age >= 35 and age < 45:
        return 35
    elif age >= 45 and age < 50:
        return 45
    elif age >= 50 and age < 56:
        return 50
    elif age >= 56:
        return 56

In [6]:
user_age = int(input("Please enter the age of the user: "))
user_occupation = int(input("Enter the occupation number: "))

user_age = map_age(user_age)

num_inputs = int(input("Enter the number of movies watched: "))
moviess = []
rat = []
u = 6041
for i in range(num_inputs):
    line = input("Enter the movie: ")
    # Split the line into MovieID, Title, and Genres
    parts = line.strip().split("::")
    title = parts[0]
    genres = parts[1].split("|")  # Split genres by "|"
    movie_id = movie_map[title]['id']
    user_input1 = float(input("Enter the movie rating: "))
    rat.append(user_input1)
    rm[u, int(movie_id)] = user_input1

In [7]:
rm = pd.DataFrame(rm)

In [8]:
i = 6040
ith_row = rm.iloc[i]
print(ith_row)

0       NaN
1       3.0
2       NaN
3       NaN
4       NaN
       ... 
3948    NaN
3949    NaN
3950    NaN
3951    NaN
3952    NaN
Name: 6040, Length: 3953, dtype: float64


In [9]:
nmpy = rm.values
list = []

num_rows, num_columns = nmpy.shape

mean = []
for v in range(num_rows):
    mean.append(np.nanmean(nmpy[v]))

/tmp/ipykernel_61983/4140226051.py:8: RuntimeWarning: Mean of empty slice
  mean.append(np.nanmean(nmpy[v]))


In [10]:
def new_pearson_correlation(X, Y, x, y):
    if(X.size == 0):
        return 0
    # Calculate the mean of X and Y
    mean_X = mean[x]
    mean_Y = mean[y]
    # Calculate the numerator
    numerator = np.sum((X - mean_X) * (Y - mean_Y))

    # Calculate the denominator
    denominator = np.sqrt(np.sum((X - mean_X) ** 2) * np.sum((Y - mean_Y) ** 2))
    if(denominator == 0):
        return 0
    # Calculate the Pearson correlation coefficient
    pearson_corr = numerator / denominator

    return pearson_corr


In [11]:
for column_index in range(num_columns):
    value = nmpy[u, column_index]
    if not np.isnan(value):
        list.append(column_index)
sim = []
for v in range(num_rows):
    ratings_u = []
    ratings_v = []
    for i in list:
        value = nmpy[v, i]
        if not np.isnan(value):
            ratings_u.append(nmpy[u, i])
            ratings_v.append(nmpy[v, i])
    ratings_u = np.array(ratings_u)
    ratings_v = np.array(ratings_v)
    sim.append(new_pearson_correlation(ratings_u, ratings_v, u, v))

In [12]:
df = pd.DataFrame(columns=['movie_index', 'rating_info'])
for movie in range(1, 3953):
    num = 0
    den = 0
    for v in range(num_rows):
        if not np.isnan(nmpy[v, movie]):
            num += sim[v] * (nmpy[v, movie] - mean[v])
            den += abs(sim[v])
    rating = mean[u]
    if(den != 0):
        rating += (num/den)
    val = nmpy[u, movie]
    if np.isnan(val):
        df = pd.concat([df, pd.DataFrame([{'movie_index': movie, 'rating_info': rating}])], ignore_index=True)
top_rated_movies = df.nlargest(5, 'rating_info')
top_rated_movies['movie_name'] = top_rated_movies['movie_index'].map(lambda x: movie_dict[str(x)]['title'])
print("Top 5 Recommended Movies for you:")
print(top_rated_movies[['movie_name']])


Top 5 Recommended Movies for you:
                           movie_name
0                    Toy Story (1995)
1                      Jumanji (1995)
2             Grumpier Old Men (1995)
3            Waiting to Exhale (1995)
4  Father of the Bride Part II (1995)
